NBA best player in each position from 2000-2019 SCRAPER

In [8]:
#Importamos las librerías que utilizaremos a lo largo de la práctica
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
import string
from requests import get
import numpy as np

#STATS
#Variaciones de la url con las estadísticas de los jugadores que exploraremos
year_url = [str(i) for i in range(2000,2019)]
position_url = ['PG', 'SG', 'SF', 'PF', 'C']

#Creamos un headers para asegurarnos de que nuestra url se escribe correctamente
headers = {"Accept-Language": "en-US, en;q=0.5"}

#Escribimos las variables que queremos encontrar para cada jugador
names = []
teams = []
years = []
positions = []
games = []
points = []
percentage = []
rebounds = []
assists = []
steals = []
blocks = []

#Para cada año de nuestro intervalo year_url:
for year in year_url:
    #Para cada posición de nuestro intervalo position_url:
    for position in position_url:
        #Solcitamos el request variando año y posición
        response = get('https://basketball.realgm.com/nba/stats/' + year +
                       '/Averages/Qualified/points/' + position + '/desc/1/Regular_Season',
                       headers = headers)
        #Analizamos el contenido de la solicitud con BeautifulSoup
        html_soup = BeautifulSoup(response.text, 'html.parser')
        #Guardamos el año que será el mismo que el que inicia el bucle
        years.append(year)
        #Guardamos la posición que será la misma que la que inicia el bucle
        positions.append(position)
        #Obtenemos las estadísticas del mejor jugador de cada año y cada posición
        #El mejor jugador es el primer container que aparece en cada página con la etiqueta <tr>
        player_rows = html_soup.tbody.findAll('tr', limit = 1)
        #Guardamos de cada etiqueta <tr> todos los td que se corresponden con las estadísticas del jugador
        player_stats = [[td.getText() for td in player_rows[i].findAll('td')]
                    for i in range(len(player_rows))]
        #Asignamos el valor correspondiente a cada estadística según la posición en la que aparece
        names.append(player_stats[0][1])
        teams.append(player_stats[0][2])
        games.append(player_stats[0][3])
        points.append(player_stats[0][22])
        percentage.append(player_stats[0][7])            
        rebounds.append(player_stats[0][18])
        assists.append(player_stats[0][19])
        steals.append(player_stats[0][20])
        blocks.append(player_stats[0][21])
    
#MVPS
#En este apartado buscamos el nombre del MVP (mejor jugador) de cada año
#Creamos la lista de MVPs
mvps=[]   
#Recorremos todos los años del bucle anterior
for year in year_url:
    #Para escribir la url correctamente, es necesario quedarnos con las dos cifras que indican el año siguiente al que inicia el bucle
    #Es decir, si estamos en 2014, year1 será '15'.
    year1 = int(year) - 2000 + 1
    #Si el valor de year1 es menor que 10, deberemos añadirle un cero delante
    #Es decir, si estamos en 2002, next_year será '03'
    if year1 < 10:
        next_year = '0'+ str(year1)
    #Si tiene dos cifras (es mayor que diez) next_year quedará igual que year1
    else:
        next_year = str(year1)
    #Solicitamos el request variando el año y el siguiente, en este caso cambiamos la página anterior a wikipedia
    response_mvp = get('https://es.wikipedia.org/wiki/Anexo:Votaci%C3%B3n_del_MVP_de_la_NBA_'+ year + '-' + next_year, headers = headers)
    #Analizamos el contenido de la solicitud con BeautifulSoup
    html_soup = BeautifulSoup(response_mvp.text, 'html.parser')
    #El MVP es el primer container que aparece en cada página con la etiqueta <td>
    mvp_container = html_soup.find_all('td')
    mvp = mvp_container[0]
    #Nos quedamos únicamente con el texto de dicho container, que será el nombre del MVP
    #Añadimos el nombre a la lista de MVPs
    mvps.append(mvp.text)
   
#Creamos una lista que nos indica si el jugador salió elegido ese año como MVP o no    
mvp_t_year=[]
#Recorremos todos los años incluidos en el primer apartado de estadísticas
for year in year_url:
    #En i nos quedamos únicamente con la última o dos últimas cifras de cada año. (2003->3 | 2014->14)
    i = int(year) - 2000
    #Con n recorremos todas las posiciones (5) 
    for n in range(len(position_url)):
        #Imponemos la condición, si en un año determinado el nombre del jugador y del MVP coinciden:
        if mvps[i]==names[i*5+n]:
            #La lista tomará le valor Yes
            mvp_t_year.append("Yes")
        else:
            #En caso contrario, tomará el valor No
            mvp_t_year.append("No")

#Recogemos en un DataFrame los datos recopilados hasta ahora            
df = pd.DataFrame({'Name': names,
                        'Team': teams,
                        'Year': years,
                        'Position': positions,
                        'Games': games,
                        'Points': points,
                        'Percentage': percentage,
                        'Rebounds': rebounds,
                        'Assists': assists,
                        'Steals': steals,
                        'Blocks': blocks,
                        'MVP': mvp_t_year})
#Ordenamos las columnas
data = ['Name', 'Team', 'Year', 'Position', 'Games', 'Points', 'Percentage', 'Rebounds', 'Assists', 'Steals', 'Blocks','MVP']
#Introducimos los cambios en las columnas
df_stats = df[data]

#ESTADÍSTICAS TOTALES
#En este apartado y el siguiente buscaremos obtener los números totales o medias de los cinco mejores jugadores de cada año en conjunto

#Partidos totales de cada quinteto
total_games = 0
#Creamos un contador n que representa el índice de los jugadores
n = 0
#Creamos la lista donde guardaremos la suma de los partidos de los cinco jugadores de cada año
totalg=[]
#Mientras n sea menor que 95 (número total de jugadores), haremos el siguiente bucle
while n < 95:
    #Para las cinco posiciones de cada año
    for i in range(len(position_url)):
        #Sumaremos los partidos de cada jugador
        total_games += int(games[i+n])
        #Cuando tengamos los cinco jugadores sumados
        if i == len(position_url)-1:
            #Añadimos ese valor a la lista
            totalg.append(total_games)
            #n aumenta en cinco cada ciclo (95/5 = 19 años)
            n += 5
            #Reiniciamos a 0 los partidos totales para el año siguiente
            total_games = 0
            
#Estadísticas medias de cada quinteto
#Nos quedamos únicamente con los atributos numéricos (puntos, porcentaje...)
stats = ['Points', 'Percentage', 'Rebounds', 'Assists', 'Steals', 'Blocks']
#Estadísticas totales
total_stats = 0
#Creamos la lista donde guardaremos las estadísticas promediadas por los cinco jugadores de cada año en conjunto
totalst=[]
#Recorremos las diferentes estadísticas de la lista
for i in stats:
    #Guardamos la columna de la estadística actual
    current_stat=df[i]
    #Creamos un contador n que representa el índice de los jugadores
    n = 0
    #Mientras n sea menor que 95 (número total de jugadores), haremos el siguiente bucle
    while n < 95:
        #Para las cinco posiciones de cada año
        for i in range(len(position_url)):
            #Sumaremos los estadísticas de cada jugador
            total_stats += float(current_stat[i+n])
            #Cuando tengamos los cinco jugadores sumados
            if i == len(position_url)-1:
                #Añadimos la media a la lista, suma entre número de jugadores (5)
                totalst.append(total_stats/5)
                #n aumenta en cinco cada ciclo (95/5 = 19 años)
                n += 5
                #Reiniciamos a 0 las estadísticas totales para el año siguiente
                total_stats = 0
            
#Agrupamos en una misma lista los partidos totales y las estadísticas promediadas de cada quinteto      
game_stats = totalg + totalst
#Creamos una lista con las líneas que añadiremos a nuestro DataFrame
#Esto es necesario ya que en nuestro apartado #STATS tenemos columnas como la posición o el equipo que quedarán vacías en los totales 
totals_per_year = []
#Recorremos cada año
for i in range(len(year_url)):
    #Creamos sublistas donde guardaremos la fila que representa el quinteto de cada año
    year_total = []
    #Recorremos las doce columnas de nuestro DataFrame
    for j in range(len(df.columns)):
        #Para la primera columna 'Name' tendremos MEDIA QUINTETO
        if j == 0:
            year_total.append("MEDIA QUINTETO")
        #Para la tercera columna 'Year' mantendremos el año 
        elif j == 2:
            year_total.append(str(2000+i))
        #Para las columnas 'Team', 'Position' y 'MVP' introduciremos un espacio en blanco
        elif j < 4 or j == 11:
            year_total.append(" ")
        #Para el resto de columnas, pondremos el valor correspondiente de la lista de totales y medias
        else:
            year_total.append(game_stats[19*(j-4) + i])
    #Añadimos la línea de cada año a la lista general
    totals_per_year.append(year_total)

#Recogemos en un DataFrame distinto los datos totales recopilados      
df_totals = pd.DataFrame(totals_per_year)   

#INSERTAR LAS ESTADÍSTICAS TOTALES
#Creamos una función que nos permite insertar las filas de las estadísticas totales en nuestro DataFrame
def Insert_row_(row_number, df, row_value): 
    #Separamos por la mitad la base de datos en dos DataFrames secundarios
    #Mitad superior
    df1 = df[0:row_number] 
    #Mitad inferior
    df2 = df[row_number:] 
    #Insertamos la fila en la mitad superior del DataFrame
    df1.loc[row_number]=row_value 
    #Unimos las dos mitades
    df_result = pd.concat([df1, df2]) 
    #Reasignamos las etiquetas de los índices
    df_result.index = [*range(df_result.shape[0])] 
    #Devolvemos el DataFrame actualizado
    return df_result

#Recorremos todas las filas del DataFrame con las estadísitcas totales
for i in range(len(df_totals)):
    #Los índices de las filas donde queremos insertar los totales serán:
    row_number = 5*(i+1) + i
    #Las filas que queremos insertar serán:
    row_value = df_totals.loc[i]
    #Recogemos en el DataFrame final el resultado
    df_stats = Insert_row_(row_number, df_stats, row_value.tolist())
    df_NBA_MVP_stats = df_stats
    
#Observamos como queda dicho DataFrame
df_NBA_MVP_stats

#Convertimos el DataFrame a CSV
df_NBA_MVP_stats.to_csv('NBA_MVP_Data.csv', header=True, index=False)

C:\Users\germa\Anaconda3\lib\site-packages\ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Name,Team,Year,Position,Games,Points,Percentage,Rebounds,Assists,Steals,Blocks,MVP
0,Gary Payton,SEA,2000,PG,82,24.2,.448,6.5,8.9,1.9,0.2,No
1,Allen Iverson,PHI,2000,SG,70,28.4,.421,3.8,4.7,2.1,0.1,Yes
2,Grant Hill,DET,2000,SF,74,25.8,.489,6.6,5.2,1.4,0.6,No
3,Karl Malone,UTA,2000,PF,82,25.5,.509,9.5,3.7,1.0,0.9,No
4,Shaquille O'Neal,LAL,2000,C,79,29.7,.574,13.6,3.8,0.5,3.0,No
5,MEDIA QUINTETO,,2000,,387,26.72,0.4882,8,5.26,1.38,0.96,
6,Stephon Marbury,NJN,2001,PG,67,23.9,.441,3.1,7.6,1.2,0.1,No
7,Allen Iverson,PHI,2001,SG,71,31.1,.420,3.8,4.6,2.5,0.3,No
8,Paul Pierce,BOS,2001,SF,82,25.3,.454,6.4,3.1,1.7,0.8,No
9,Chris Webber,SAC,2001,PF,70,27.1,.481,11.1,4.2,1.3,1.7,No
